In [9]:
import torch
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# 多输入多输出通道

## 多个输入通道

- 彩色图像可能有RGB三个通道，转换为灰度会丢失信息


![](channel0.png)

- 当添加通道时，输入和隐藏的表示都变成了三维张量
    - 例如，每个RGB输入图像具有$3\times h\times w$的形状
- 将这个大小为$3$的轴称为**通道**（channel）维度

\begin{problem}\label{prob:channelsKernel}
当存在多个输入通道时，如何计算卷积？
\end{problem}


- 当输入包含多个通道时，需要构造一个与输入数据具有相同输入通道数的卷积核，以便与输入数据进行互相关运算

- 假设输入的通道数为$c_i$，那么卷积核的输入通道数也需要为$c_i$
- 如果卷积核的窗口形状是$k_h\times k_w$。当$c_i>1$时，卷积核的每个输入通道将包含形状为$k_h\times k_w$的张量

- 这些张量$c_i$连结在一起可以得到形状为$c_i\times k_h\times k_w$的卷积核

- 由于输入和卷积核都有$c_i$个通道，可以对每个通道输入的二维张量和卷积核的二维张量进行互相关运算，再对通道求和（将$c_i$的结果相加）得到二维张量

\begin{example}\label{example:twoChannels}
一个具有两个输入通道的二维互相关运算的示例
\end{example}


![](channel1.png)

用公式表达
- 输入$\mathbf{X}$： $c_i × n_h \times n_w$
- 核$\mathbf{W}$：$c_i × k_h × k_w$，偏移这里没有写，是个长度为$c_i$的一维向量。
- 输出$\mathbf{Y}$：$m_h × m_w$，**无论输入通道有多少，都是单输出通道**。

$$Y = \sum^{c_i}_{i=0} X_{i,:,:}★ W_{i,:,:}$$

In [4]:
# 用代码实现多输入通道

def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(corr2d(x, k) for x, k in zip(X, K))

In [3]:
def corr2d(X, K):
    """计算二维互相关运算
        X是输入张量
        K是卷积核张量
    """
    h, w = K.shape # 核的形状
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1)) # 输出的形状
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum() # 卷积
    return Y

- 构造与上图中的值相对应的输入张量`X`和核张量`K`

In [10]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

print(f'输入张量X {X}')

print(f'核K {K}')

输入张量X tensor([[[0., 1., 2.],
         [3., 4., 5.],
         [6., 7., 8.]],

        [[1., 2., 3.],
         [4., 5., 6.],
         [7., 8., 9.]]])

核K tensor([[[0., 1.],
         [2., 3.]],

        [[1., 2.],
         [3., 4.]]])

In [11]:
corr2d_multi_in(X, K)


tensor([[ 56.,  72.],
        [104., 120.]])

## 多个输出通道

- 可以有多个三维卷积核，每个核生成一个输出通道

- 输入$\mathbf{X}$：$c_i × n_h \times n_w$，与之前写法相同
- 核$\mathbf{W}$：$c_o \times c_i × k_h × k_w$，注意这里多了$c_o$
- 输出$\mathbf{Y}$：$c_o \times m_h × m_w$
 
$$Y_{i,:,:} = X ★ W_{i,:,:,:} \ \ for \ i = 1,... c_o$$

- 每个输出通道可以识别特定模式

![](channel2.png)

\begin{example}\label{example:multioutput}
实现一个计算多个通道的输出的互相关函数
\end{example}

In [14]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

- ```python
    torch.stack(tensors, dim=0, *, out=None) → Tensor
```

    - 沿着一个新维度连接一系列张量
    - 所有张量必须有相同的形状

In [12]:
# 通过将核张量K与K+1（K中每个元素加1）和K+2连接起来，构造了一个具有3个输出通道的卷积核

K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [15]:
# 对输入张量X与卷积核张量K执行互相关运算。现在的输出包含3个通道

corr2d_multi_in_out(X, K)


tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## $1 \times 1$卷积层

- $k_h=k_w =1$是一个受欢迎的选择。它不识别空间模式，只是融合通道。

- $1\times 1$卷积失去了卷积层的特有能力——在高度和宽度维度上，识别相邻元素间相互作用的能力

\begin{example}\label{example:1x1kernel}
下图为例：输入3个神经元，输出2个神经元
\end{example}

![](11kernel.png)

输入和输出具有相同的高度和宽度，输出中的每个元素都是从输入图像中同一位置的元素的线性组合

- 相当于输入形状为$c_i\times n_hn_w$，权重为$c_o×c_i$的全连接层

In [16]:
# 使用全连接层实现1x1卷积。注意，需要对输入和输出的数据形状进行调整

def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

当执行$1\times 1$卷积运算时，上述函数相当于先前实现的互相关函数`corr2d_multi_in_out`

In [17]:
# 用数据检验1x1卷积运算与互相关函数的一致

X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

In [23]:
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
float(torch.abs(Y1 - Y2).sum())

1.1615920811891556e-06

# 二维卷积层

- 输入**X**: $c_i × n_h \times n_w$
- 核**W**: $c_o \times c_i × k_h × k_w$
- 偏差 **B**: $c_o\times c_i$
- 输出**Y**: $c_o \times m_h × m_w$
$$Y = X ★ W + B$$
- 计算复杂度（浮点计算数FLOP） $O(c_ic_ok_hk_wm_hm_w)$ 
    - [注意]复杂度只关心乘法运算量
    - FLOPS(Floating-point Operations Per Second)为每秒所执行的浮点运算次数。它是一个衡量计算机计算能力的量，经常使用在那些需要大量浮点运算的科学运算中。
$$c_i=c_o=100, k_h=h_w=5, m_h=m_w=64 ==>> 1G(FLOP) $$
- 10层，1M样本，10PFlops
    - CPU：0.15G/s， TF=18h
    - GPU：12G/s， TF=14min